In [ ]:
#Imports 
#-----------------------------
# bring in the csv's to work with using pandas dataframe 
#-----------------------------


# pandas info http://pandas.pydata.org/


import numpy as np
import pandas as pd
import csv

keys= pd.read_csv("adjData/popData2.csv")
adjList= pd.read_csv("adjData/nlist.csv")
isin=pd.read_csv("adjData/isin2.csv")

keyList= keys[['GEOID']]
# two step join to get only maryland-maryland census tracts 
step= pd.merge(adjList,keyList,how='inner',left_on='NEIGHBOR_TRACTID',right_on='GEOID')
adjSet= pd.merge(step,keyList,how='inner',left_on='SOURCE_TRACTID',right_on='GEOID')
keys=pd.merge(keys,isin,how='inner',left_on='GEOID',right_on='GEOID')

In [ ]:
#PRECOMPUTE NEIGHBORS
#-----------------------------
# precompute number of neighbors so we do not leave any adjacent districts this is to make our lives way easier later 
#-----------------------------

# checked maryland values for no zero values so we can look for orphans right after they are created 

partners={}
for i in range(1,keyList.shape[0]):#
    key = keyList.iloc[i]['GEOID']
    value = adjSet[adjSet.SOURCE_TRACTID == key] # adjacent tracts for census tracts key in NEIGHBOR_TRACTID col 
    partners[key] = value.shape[0] 
   

In [ ]:
# choose a starting discrict and start working 
# for test purposes we are going to make 1 district 24029950100
# could we precomput the number of districts that are adjcant to a disctric and each time they are added subract one -- much faster and
# we can search easily 

#globals
#feasibility constraint so we can make manual changes 
targetPop=np.int64(625740)#625741)
#function defs
def findDistrict(adjacent , tracts, pop, det ):
    #Check to see if we are done 
    
    if (pop+det >= targetPop).all():
        return tracts
    # choose next district based on selection method
    
    # get adjacent with most edge 
    v=list(adjacent.values())
    k=list(adjacent.keys())
    # index of tract with most neighbors
    toAdd =k[v.index(max(v))]
    
    #print(toAdd)
    # once selection is made addtract will handle all of the set updates 
    adjacent,tracts,pop = addTract(adjSetl=adjacent,tract=toAdd,dist=tracts,pop=pop)
    #print(tracts)
    # recursice call
    if  (len ( adjacent ) == 0 ):
        print('error!!')    
        return (tracts,pop) 
    #print (tracts)
    return findDistrict(adjacent,tracts,pop,det)

def addTract (adjSetl,tract,dist,pop):
   
    # we need to get the index of a given tract  
    index=(keys[keys.GEOID == tract])[['index']].iloc[0]
    #print('-----ISIN------')
    #print (keys.iloc[index]['IsIn'])
    if (keys.iloc[index]['IsIn'].any() == 0 ):
        print('added')
        #add tract to the district 
        dist.append(tract)
        #remove dist from adjset 
        adjSetl[tract] =-1
        keys.set_value(index, 13 , 1,takeable=True)
        # update pop
        pop=pop+(keys[keys.GEOID == tract])[['POP100']].iloc[0]  
        #update adj set for new adjacent 
        adjSetl=updateAdjSet(localAdjSet=adjSetl,index=tract)
        # deal with oprhaned districts 
        orphans=isOrphan(tract=tract)
        if (orphans != []):
            #deal with new oprhans we are going to create by adding this new tract 
            #being lazy / greedy and i am just going to add them this might get ~ weird 
            for val in orphans:
                #addTract(adjSetl=adjSetl,tract=val,dist=dist,pop=pop)
                afaf=val
    else: 
        print ('trying to add already added')
        v=list(adjSetl.values())
        k=list(adjSetl.keys())
        toDel =k[v.index(max(v))]
        del adjSetl[toDel]
        
    return (adjSetl,dist,pop)

def isOrphan (tract):
    
    return []

def updateAdjSet(localAdjSet,index):
    toAdd = adjSet[adjSet.SOURCE_TRACTID == index] # adjacent tracts for census tracts key in NEIGHBOR_TRACTID col 
    for i in range(1,toAdd.shape[0]):#
        key = toAdd.iloc[i]['NEIGHBOR_TRACTID']   
        if  key in localAdjSet:
            if (localAdjSet[key]==-1):
                # do nothing this value is alredy been added to the district 
                b=1
            else:
                localAdjSet[key]=localAdjSet[key]+1
        else:
            localAdjSet[key]=1       
    return localAdjSet




In [ ]:
a={};
#get the first District 

startTractId1 = 50027965400
first= [startTractId1]
firstStartPop=(keys[keys.GEOID == startTractId1])[['POP100']].iloc[0]
locAdjSet1=updateAdjSet(localAdjSet=a,index=startTractId1)
first= findDistrict(adjacent=locAdjSet1,tracts=first,pop=firstStartPop,det=0)
print(first)

In [ ]:
#begin the recursive fixing 
#pd.set_option('display.max_rows', 1000)
import math as math
# now we have the subset for which to work with 
lo=keys[keys.IsIn == 0 ].GEOID  

def fix (tracts,toWrite):
    returnFlag=0;
    for row in tracts:
        toCheck=adjSet[adjSet.SOURCE_TRACTID == row].NEIGHBOR_TRACTID
        adjVal=0
        n=toCheck.shape
        scalar=n[0];
        adjacent={}
        for elm in toCheck:
            if elm in first:
                key=1
            
            # deal with the dictionary values
            elif elm in toWrite:
                key=toWrite[elm] 
            else:
                key=-1
            
            if  key in adjacent and key != -1:
                adjacent[key]=adjacent[key]+1
            elif key != -1:
                adjacent[key]=1     
        #compute the value and update 
        # get adjacent with most edge 
    
        v=list(adjacent.values())
        k=list(adjacent.keys())
        if v == []:
            writeme=0
        # index of tract with most neighbors
        else:
            writeme =k[v.index(max(v))]
        # now we have the district to put our tract in 
        if (writeme != 0 ):
            toWrite[row]=writeme
            # set the tracts to is in in keys. 
            toUpdate=keys[keys.GEOID == row].index
            keys.set_value(toUpdate[0], 13 , 1,takeable=True)
        else:
            print('island found')
            returnFlag=1;
    #end of loop ----
    toWrite[24023000100]=5
    toWrite[24023000200]=5
    return toWrite
    #fix(lo,toWrite)
# end of function 


toWrite ={}
fix(lo,toWrite)
print('donezo')

In [ ]:
#generate the CSV File for raw output 
f = open("firstVT.csv",'w')
f.write('GEOID' +','+'color' + '\n' )
for row in first:
    f.write( str(row) +','+'1' + '\n' )
f.close()


In [ ]:
#generate the CSV File for filtered output 
f = open("secondVT.csv",'w')
f.write('GEOID' +','+'color' + '\n' )
for row in first:
    f.write( str(row) +','+'1' + '\n' )
for key in toWrite.keys():
     f.write( str(key) +','+ str(toWrite[key]) + '\n' )
f.close()

In [ ]:
a=1
a =a-1
print(a)

In [ ]:
print (first)

In [ ]:
print(len(first))
print(len(keyList))
print(len(toWrite))

In [ ]:
print(keys)